# MNIST-Distributed-Backdoor
### Image Translation

#### Imports

In [2]:
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.optim import *
from cox.utils import Parameters

from robustness import model_utils, datasets, train, defaults
from robustness.model_utils import make_and_restore_model
from DatasetsNew import MNIST

import cox.store
import torch as ch
import DatasetsNew
import defaultsNew
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import os
import cv2
import copy
import numpy as np

import pickle
import sys
import psutil
import gc


from label_maps import CLASS_DICT
from user_constants import DATA_PATH_DICT

#### DATASET CONSTANTS 

In [9]:
NUM_WORKERS = 8
BATCH_SIZE = 100

In [10]:
DATA = 'MNIST' 

#### Load dataset

In [13]:
# Load dataset
dataset_function = getattr(DatasetsNew, 'MNIST')
DATA_PATH_DICT[DATA]
dataset = dataset_function(DATA_PATH_DICT[DATA])
_, test_loader = dataset.make_loaders(workers=NUM_WORKERS,
                                      batch_size=BATCH_SIZE, 
                                      data_aug=False)
data_iterator = enumerate(test_loader.dataset)

==> Preparing dataset mnist..


In [14]:
model_backdoored, _ = make_and_restore_model(arch='resnet18', dataset=dataset,resume_path='models/MNIST_small_trigger_Distributed_100_epochs_checkpoint.pt.best',parallel=False) 
model_backdoored.eval()
print()

=> loading checkpoint 'models/MNIST_small_trigger_Distributed_100_epochs_checkpoint.pt.best'
=> loaded checkpoint 'models/MNIST_small_trigger_Distributed_100_epochs_checkpoint.pt.best' (epoch 66)



In [15]:
_, (imgs, label) = next(enumerate(test_loader))

In [16]:
targ = []
targ_lbl = 0
for i in range(BATCH_SIZE):
    targ.append(targ_lbl)
targ = ch.tensor(targ)

In [17]:
kwargs = {
        #'criterion': ch.nn.CrossEntropyLoss(reduction='none'),
        'constraint':'2',
        'eps': 100,
        'step_size': 0.25,
        'iterations': 150,
        'do_tqdm': True,
        'targeted': True,
}

### Translation for Distributed Backdoored MNIST model

In [18]:
_, train_loader = dataset.make_loaders(workers=NUM_WORKERS, batch_size=BATCH_SIZE)
_, (imgs, label) = next(enumerate(train_loader))

for targ_lbl in range(10):
    targ = []


    for i in range(BATCH_SIZE):
        targ.append(targ_lbl)
    targ = ch.tensor(targ)

    _, img_translated = model_backdoored(imgs.cuda(), targ.cuda(), make_adv=True, **kwargs)

    for i in range(4):
        _, (imgs, label) = next(enumerate(train_loader))
        targ = []
        for i in range(BATCH_SIZE):
            targ.append(targ_lbl)
        targ = ch.tensor(targ)

        _, img_translated_new = model_backdoored(imgs.cuda(), targ.cuda(), make_adv=True, **kwargs)
        img_translated = ch.cat((img_translated, img_translated_new), 0)

    print(img_translated.size())
    
    
    filename = "saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_" + str(targ_lbl) + ".pkl"
    
    with open(filename, 'wb') as handle:
        pickle.dump(img_translated, handle)
    print("Saved: ", filename)

==> Preparing dataset mnist..


Current loss: 9.054082870483398:   2%|▏         | 3/150 [00:00<00:07, 20.26it/s]

torch.Size([500, 1, 28, 28])
Saved:  saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_0.pkl


Current loss: 8.242544174194336:   1%|▏         | 2/150 [00:00<00:07, 19.42it/s]

torch.Size([500, 1, 28, 28])
Saved:  saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_1.pkl


...

torch.Size([500, 1, 28, 28])
Saved:  saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_7.pkl


Current loss: 8.533876419067383:   2%|▏         | 3/150 [00:00<00:07, 20.42it/s]

torch.Size([500, 1, 28, 28])
Saved:  saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_8.pkl


Current loss: 1.2111663636460435e-06: 100%|██████████| 150/150 [00:07<00:00, 19.74it/s]

torch.Size([500, 1, 28, 28])
Saved:  saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_9.pkl
